In [412]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import Counter
import math
import re
import pandas as pd

THE TOKENIZER CLASS IS BASICALLY USED TO DO PREPROCESSING LIKE TOKENOZING (DETECTING UNIQUE WORDS), STOP WORDS REMOVAL, REGEX(SPECIAL CHARACTERS REMOVAL), REMOVING ARTICLES (SINCE THEY ARE ABUNDANT AND DO NOT ADD TO THE MEANING OF THE FILE), REMOVING EXCESS WHITE SPACE, APOSTROPHES.

NOTE: SINCE NLP BUILT LIBRARIES ARE NOT ALLOWED I HAVE NOT USED LEMMETIZATION TO FIND LEMMAS OF SIMILAR WORDS 

In [476]:
class tokenizer:
    def tokenize(self,sentences):
        for i in sentences:
            tokens = sentences.split()
            return tokens
    
    def stop_words_removal(self,sentences):
        new_text=""
        tokens=[i.lower() for i in sentences]
        list_stop_words=['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
        stop_words_removed_string=[word for word in tokens if word not in list_stop_words]
        for i in stop_words_removed_string:
            new_text=new_text+" "+i
        return new_text
    
    def regex1(self,sentences):
        removeSpecialChars = sentences.translate ({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
        return removeSpecialChars
    
    def remove_articles(self,sentences):
        removeArticles = re.sub('\s+(a|an|and|the)(\s+)', '\2', sentences)
        return removeArticles

    def excess_white_space_removal(self,sentences):
        return " ".join(sentences.split())
    
    def apostrophes(self,sentences):
        return sentences.replace("'","")

NOW WE FIRST MAKE A count_terms_dictionary(DICTIONARY CONTAINING UNIQUE TERMS ALONG WITH ITS FREQUENCY OF OCCURANCE), computeTF(COMPUTES THE FREQUENCY OF EACH TERM WITH RESPECT TO LENGTH OF THE STRING), computeIDF(COMPUTES WHETHER THE TERM IS COMMON OR RARE IN A GIVEN DOCUMENT), computeTFIDF (BASICALLY MULTIPLIES TERM FREQUENCY WITH INVERSE DOCUMENT FREQUENCY)

In [477]:
class tf_idf1:
    def count_terms_dictionary(self,sentences):
        DF={}
        words=sentences.split()
        for i in range(len(words)):
            if words[i] not in DF:
                DF[words[i]]=1
            else:
                DF[words[i]]+=1
        return DF
        
    def computeTF(self,sentences):
        tfDict={}
        s1=tf_idf1()
        counter=s1.count_terms_dictionary(sentences)
        words_count=len(sentences.split())
        for w,c in counter.items():
            tfDict[w]=c/float(words_count)
        return tfDict
    
    def computeIDF(self,sentences):
        s1=tf_idf1()
        idfDict={}
        N=len(sentences.split())
        idfDict=s1.computeTF(sentences)
        for w,c in idfDict.items():
            idfDict[w] = math.log10(N/(float(c)+1))
            
        return idfDict
    
    def computeTFIDF(self,sentences):
        s1=tf_idf1()
        tfidf={}
        counter=s1.computeTF(sentences)
        idfs=s1.computeIDF(sentences)
        for w,c in counter.items():
            tfidf[w]=c*idfs[w]
        return tfidf

CALCULATE IS BASICALLY A PIPELINE FOR CALLING ALL THE ABOVE FUNCTIONS IN ORDER

In [478]:
class calc:
    def calculate(self,sent):
        s=tokenizer()
        s1=tf_idf1()
        sent1=s.tokenize(sent)
        sent2=s.stop_words_removal(sent1)
        sent3=s.regex1(sent2)
        sent4=s.remove_articles(sent3)
        sent5=s.excess_white_space_removal(sent4)
        sent6=s.apostrophes(sent5)
        tfidfs=s1.computeTFIDF(sent6)
        return tfidfs

THE SCORE FUNCTION IS USED TO SCORE THE DOCUMENTS BASED ON COSINE DISTANCE. THE SCORE EXISTS BETWEEN 0 AND 1

In [479]:
class scoring:
    def score(self,a,b):
        c=pd.DataFrame([a,b])
        k=c.fillna(0)
        k1=k.iloc[0]
        k2=k.iloc[1]
        dot_prod=np.dot(k1,k2)
        squares1=0
        squares2=0
        squares1=[i*i for i in k1]
        sum_squares1=sum(squares1)
        mag_squares1=np.sqrt(sum_squares1)
        squares2=[i*i for i in k2]
        sum_squares2=sum(squares2)
        mag_squares2=np.sqrt(sum_squares2)
        cosine=dot_prod/(mag_squares1*mag_squares2)
        return cosine

THE MAIN FUNCTION IS USED TO CALCULATE THE SCORES FOR TWO DIFFERENT TEXTS

In [492]:
class main_function:
    def run(self,text1,text2):
        t1=calc()
        t2=scoring()
        tfidf_1=t1.calculate(text1)
        tfidf_2=t1.calculate(text2)
        scores=t2.score(tfidf_1,tfidf_2)
        return scores

In [498]:
text1="The easiest way to earn points with Fetch Rewards is to just shop for the products you already love. If you have any participating brands on your receipt, you'll get points based on the cost of the products. You don't need to clip any coupons or scan individual barcodes. Just scan each grocery receipt after you shop and we'll find the savings for you."
text2="The easiest way to earn points with Fetch Rewards is to just shop for the items you already buy. If you have any eligible brands on your receipt, you will get points based on the total cost of the products. You do not need to cut out any coupons or scan individual UPCs. Just scan your receipt after you check out and we will find the savings for you."

In [499]:
l1=main_function()
final_score=l1.run(text1,text2)
print(final_score)

0.7924320151575729
